In [2]:
import undetected_chromedriver as uc
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ec
import time
from openpyxl import Workbook


def captcha_solver():
    try:
        while True:
            element = driver.find_element_by_css_selector('#px-captcha')
            action = ActionChains(driver)
            click = ActionChains(driver)
            action.click_and_hold(element)
            action.perform()
            time.sleep(10)
            action.release(element)
            action.perform()
            time.sleep(0.2)
            action.release(element)
    except:
        return None

driver = uc.Chrome(executable_path=ChromeDriverManager().install())
driver.get('https://www.zillow.com/homes/_rb/')
driver.implicitly_wait(1)
captcha_solver()

addresses = ['13503 BOHANNON AVE', '6908 WYTHE HILL CIR', '10518 OLD TAYLORSVILLE RD', '6610 GLENDALE RD', '2001 EASTBRIDGE CT', '717 LAMPTON ST', '1421 BERRY BLVD', '3930 GREENHURST DR', '8207 MONTERAY VILLAGE LN']
lst = []

for a in addresses:
    bed, bath, sqft, status, zes, date, event, price = '-', '-', '-', '-', '-', '-', '-', '-'
    try:
        inp = driver.find_element_by_tag_name('input')
        inp.send_keys(Keys.CONTROL, 'A')
        inp.send_keys(Keys.DELETE)
        inp.send_keys(a)
        inp.send_keys(Keys.ENTER)
        captcha_solver()
        
        try:
            details = WebDriverWait(driver, 2).until(ec.visibility_of_element_located((By.CLASS_NAME, "summary-container")))
        except:
            details = WebDriverWait(driver, 2).until(ec.visibility_of_element_located((By.CLASS_NAME, "ds-home-details-chip")))
            
            
        for i in details.text.split('\n'):
            if 'bd' in i:
                bed = i.split('bd')[0][-2]
                bath = i.split('bd')[-1].split(' ')[0]
                sqft = i.split('ba')[-1].split(' ')[0]
            if 'Zestimate®:' in i:
                status, zes = i.split('Zestimate®: ')[0], i.split('Zestimate®: ')[1]
                if 'None' in zes:
                    zes = 'None'
                if 'Rent' in zes:
                    zes = zes.replace('Rent', '')
                if 'assessed' in zes:
                    zes = zes.split(' ')[0]
                if 'Sold' in status:
                    status = 'Sold'
                    
            
        try:    
            try:
                driver.find_element_by_link_text('Neighborhood details').click()
            except:
                driver.find_element_by_link_text('Price and tax history').click()
            price_history = driver.find_elements_by_tag_name('tr')[1].text.split(' ')
            date, event, price = price_history[0], price_history[1], price_history[2]
        except:
            pass

    
        driver.back()
        print(f"""ADDRESS: {a}
        Bed: {bed}
        Bath: {bath}
        Area: {sqft}
        Status: {status}
        Zestimate: {zes}
        Date: {date}
        Event: {event}
        Price: {price}
        """)
    except:
        try:
            WebDriverWait(driver, 2).until(ec.visibility_of_element_located(driver.find_element_by_css_selector('#px-captcha')))
            captcha_solver()
        except:
            driver.get('https://www.zillow.com/homes/_rb/')
            captcha_solver()
        
        
    lst.append({'address': a,
    'bed': bed,
    'bath': bath,
    'area': sqft,
    'status': status,
    'zestimate': zes,
    'date': date,
    'event': event,
    'price': price})    
        
        
workbook = Workbook()
sheet = workbook.active

sheet["A1"] = "Address"
sheet["B1"] = "Bed"
sheet["C1"] = "Bath"
sheet["D1"] = "Area"
sheet["E1"] = "Status"
sheet["F1"] = "Zestimate"
sheet["G1"] = "Date"
sheet["H1"] = "Event"
sheet["I1"] = "Price"

for i in range(len(lst)):
    sheet[f"A{i+2}"] = lst[i]['address']
    sheet[f"B{i+2}"] = lst[i]['bed']
    sheet[f"C{i+2}"] = lst[i]['bath']
    sheet[f"D{i+2}"] = lst[i]['area'] + ' sqft'
    sheet[f"E{i+2}"] = lst[i]['status']
    sheet[f"F{i+2}"] = lst[i]['zestimate']
    sheet[f"G{i+2}"] = lst[i]['date']
    sheet[f"H{i+2}"] = lst[i]['event']
    sheet[f"I{i+2}"] = lst[i]['price']
    
workbook.save(filename="Zillow.xlsx")

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 97.0.4692
[WDM] - Get LATEST driver version for 97.0.4692
[WDM] - Driver [C:\Users\fagho\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache


ADDRESS: 13503 BOHANNON AVE
        Bed: -
        Bath: 1
        Area: 886
        Status: Pre-foreclosure
        Zestimate: None
        Date: 8/17/2017
        Event: Sold
        Price: $8,500
        
ADDRESS: 6908 WYTHE HILL CIR
        Bed: 3
        Bath: 3
        Area: 2,881
        Status: Off market
        Zestimate: $371,600 
        Date: 11/27/2018
        Event: Sold
        Price: $243,900
        
ADDRESS: 6610 GLENDALE RD
        Bed: 3
        Bath: 3
        Area: 1,923
        Status: Sold
        Zestimate: $381,100
        Date: 1/6/2022
        Event: Sold
        Price: $380,000
        
ADDRESS: 2001 EASTBRIDGE CT
        Bed: -
        Bath: 1
        Area: 1,665
        Status: Rent 
        Zestimate: $1,3762020
        Date: 2/26/2001
        Event: Sold
        Price: $119,250
        
ADDRESS: 717 LAMPTON ST
        Bed: 3
        Bath: 2
        Area: 2,315
        Status: Sold
        Zestimate: $283,700
        Date: 12/3/2021
        Event: Sold
